# Personalization Model

Inputs:
- User profiles: Collected from user registration and account information.
- Preferences: Gathered from user settings and preferences.
- Past behavior: Derived from historical interaction data and usage patterns.
- Demographic data: Collected during user registration and from public records.

Outputs:
- Personalized recommendations: Travel suggestions based on user-specific data.
- Itineraries: Customized travel plans tailored to individual preferences.
- Notifications: Alerts and updates based on user interests and activities.

Usage:
- Enhances user experience by providing customized travel suggestions and alerts based on user-specific data.

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

Generating Sample Data

In [4]:
np.random.seed(42)
users = ['User1', 'User2', 'User3', 'User4', 'User5']
items = ['ItemA', 'ItemB', 'ItemC', 'ItemD', 'ItemE']

user_profiles = {
    'User': users,
    'Age': np.random.randint(18, 70, size=5),
    'Gender': np.random.choice(['M', 'F'], size=5),
    'Location': np.random.choice(['NY', 'LA', 'SF', 'TX', 'FL'], size=5),
    'PreferenceA': np.random.choice([0, 1], size=5),
    'PreferenceB': np.random.choice([0, 1], size=5),
    'PreferenceC': np.random.choice([0, 1], size=5),
}

user_behavior = {
    'User': np.random.choice(users, size=20),
    'Item': np.random.choice(items, size=20),
    'Rating': np.random.randint(1, 6, size=20),
    'Timestamp': pd.date_range(start='2024-01-01', periods=20, freq='D')
}

df_profiles = pd.DataFrame(user_profiles)
df_behavior = pd.DataFrame(user_behavior)

print("User Profiles:")
print(df_profiles)
print("\nUser Behavior:")
print(df_behavior)

User Profiles:
    User  Age Gender Location  PreferenceA  PreferenceB  PreferenceC
0  User1   56      F       SF            1            1            1
1  User2   69      M       SF            1            1            1
2  User3   46      M       SF            0            1            1
3  User4   32      M       FL            1            1            0
4  User5   60      F       TX            0            1            0

User Behavior:
     User   Item  Rating  Timestamp
0   User4  ItemB       2 2024-01-01
1   User2  ItemD       2 2024-01-02
2   User5  ItemA       4 2024-01-03
3   User4  ItemD       5 2024-01-04
4   User1  ItemB       2 2024-01-05
5   User1  ItemB       2 2024-01-06
6   User3  ItemA       4 2024-01-07
7   User3  ItemB       2 2024-01-08
8   User2  ItemE       2 2024-01-09
9   User4  ItemB       4 2024-01-10
10  User4  ItemD       4 2024-01-11
11  User3  ItemD       1 2024-01-12
12  User4  ItemD       5 2024-01-13
13  User4  ItemD       5 2024-01-14
14  User1  Item

In [6]:
# Create user-item interaction matrix
user_item_matrix = pd.pivot_table(df_behavior, index='User', columns='Item', values='Rating', fill_value=0)

user_item_matrix

Item,ItemA,ItemB,ItemC,ItemD,ItemE
User,,,,,
User1,0.0,2.0,0.0,4.00,2.0
User2,0.0,0.0,0.0,2.00,2.0
User3,4.0,2.0,5.0,1.00,0.0
User4,0.0,3.0,0.0,4.75,0.0
User5,3.0,4.0,0.0,0.00,0.0


In [9]:
# Convert to sparse matrix format
sparse_user_item = csr_matrix(user_item_matrix.values)
print(sparse_user_item)

  (0, 1)	2.0
  (0, 3)	4.0
  (0, 4)	2.0
  (1, 3)	2.0
  (1, 4)	2.0
  (2, 0)	4.0
  (2, 1)	2.0
  (2, 2)	5.0
  (2, 3)	1.0
  (3, 1)	3.0
  (3, 3)	4.75
  (4, 0)	3.0
  (4, 1)	4.0


Train the Model

In [10]:
# Initialize the ALS model
model = AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20)

# Train the model
model.fit(sparse_user_item.T)

100%|██████████| 20/20 [00:00<00:00, 1033.60it/s]


In [15]:
# Function to get top N recommendations for a given user
def get_top_n_recommendations(user, n=5):
    if user not in user_item_matrix.index:
        raise ValueError(f"User '{user}' not found in user-item matrix.")
    # To get index of user
    user_index = user_item_matrix.index.get_loc(user)
    # Recommend items to the user
    recommendations = model.recommend(user_index, sparse_user_item[user_index], N=n)
    # Extract recommended items
    top_n_recommendations = [user_item_matrix.columns[i] for i in recommendations[0]]
    
    return top_n_recommendations

# Example usage
user = 'User1'
top_n_recommendations = get_top_n_recommendations(user)
print(f'Top recommendations for {user}: {top_n_recommendations}')

Top recommendations for User1: ['ItemC', 'ItemA', 'ItemE', 'ItemD', 'ItemB']


In [16]:
# Encode categorical features
le_location = LabelEncoder()
df_profiles['Location'] = le_location.fit_transform(df_profiles['Location'])
le_gender = LabelEncoder()
df_profiles['Gender'] = le_gender.fit_transform(df_profiles['Gender'])

# Standardize and reduce dimensionality of the user profiles
scaler = StandardScaler()
profile_features = ['Age', 'Gender', 'Location', 'PreferenceA', 'PreferenceB', 'PreferenceC']
df_profiles_scaled = scaler.fit_transform(df_profiles[profile_features])
pca = PCA(n_components=3) # used if more features are present
df_profiles_pca = pca.fit_transform(df_profiles_scaled)

In [18]:
df_profiles_pca

array([[ 0.43880437,  0.89830266,  1.07400689],
       [-0.0321654 ,  1.71603512, -0.10449137],
       [-0.141424  , -0.05248103, -1.68030469],
       [-2.53245766, -1.13797661,  0.48415666],
       [ 2.26724269, -1.42388014,  0.22663251]])

In [19]:
# Adjust recommendation scores based on user profiles and preferences
def adjust_recommendations(user, recommendations):
    user_profile = df_profiles[df_profiles['User'] == user].iloc[0]
    user_vector = pca.transform(scaler.transform([user_profile[profile_features]]))[0]
    preference_scores = []

    for item in recommendations:
        score = 0
        if user_profile['PreferenceA']:
            score += np.random.rand() 
        if user_profile['PreferenceB']:
            score += np.random.rand()
        if user_profile['PreferenceC']:
            score += np.random.rand()
        preference_scores.append(score)
    
    adjusted_recommendations = sorted(zip(recommendations, preference_scores), key=lambda x: x[1], reverse=True)
    adjusted_recommendations = [rec[0] for rec in adjusted_recommendations]
    return adjusted_recommendations

# Example usage with adjusted recommendations
adjusted_recommendations = adjust_recommendations(user, top_n_recommendations)
print(f"Adjusted recommendations for {user}: {adjusted_recommendations}")

Adjusted recommendations for User1: ['ItemD', 'ItemA', 'ItemC', 'ItemE', 'ItemB']


In [20]:
# Function to provide personalized recommendations, itineraries, and notifications
def provide_personalized_experience(user):
    top_n_recommendations = get_top_n_recommendations(user)
    adjusted_recommendations = adjust_recommendations(user, top_n_recommendations)
    
    itinerary = f"Suggested Itinerary for {user}: Visit {', '.join(adjusted_recommendations[:3])}."
    notifications = f"Notifications for {user}: Special offers available for {', '.join(adjusted_recommendations)}."
    
    return {
        'User': user,
        'Recommendations': adjusted_recommendations,
        'Itinerary': itinerary,
        'Notifications': notifications
    }

# Example usage
user_experience = provide_personalized_experience(user)
print(f"Personalized Experience for {user}:\n")
print(f"Recommendations: {user_experience['Recommendations']}")
print(f"Itinerary: {user_experience['Itinerary']}")
print(f"Notifications: {user_experience['Notifications']}")

Personalized Experience for User1:

Recommendations: ['ItemE', 'ItemB', 'ItemC', 'ItemD', 'ItemA']
Itinerary: Suggested Itinerary for User1: Visit ItemE, ItemB, ItemC.
Notifications: Notifications for User1: Special offers available for ItemE, ItemB, ItemC, ItemD, ItemA.
